## Data preparation for CHD

In [ ]:
# import libraries
import glob
import re
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from scipy.cluster.hierarchy import ward, dendrogram, fcluster
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
import networkx as nx
from ast import literal_eval
import warnings
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from pmdarima.arima import auto_arima
from statsmodels.graphics.tsaplots import plot_acf
warnings.filterwarnings('ignore')

In [ ]:
def csv_to_dataframe(folder_path):
    """
    Read all the csv files in the given folder and concatanate them into a single dataframe
    """
    df_list = []
    csv_files = glob.glob(folder_path)

    for filename in csv_files:
        df = pd.read_csv(filename)
        df_list.append(df)
    
    combined_df = pd.concat(df_list, ignore_index=True)
    return combined_df


In [ ]:
# folder path all inpatient csv's
inp_folder_path = "data/INPT 2015-2023 Q1-Q3/*.csv"
# folder path all emergency csv's
ed_folder_path = "data/ED 2015-2023Q1-Q3/*.csv"

inp_df = csv_to_dataframe(inp_folder_path)
ed_df = csv_to_dataframe(ed_folder_path)

In [ ]:
# print the dimensions
print(f"inpatient data: {inp_df.shape}")
print(f"outpatient/ed data: {ed_df.shape}")

In [ ]:
# ICD-10 CM codes for CHD
chd_icd_10_list = ["I25", "I25.1", "I25.10", "I25.11", "I25.110", "I25.111", "I25.112", "I25.118", "I25.119", 
                   "I25.2", "I25.3", "I25.4", "I25.41", "I25.42", "I25.5", "I25.6", "I25.7", "I25.70", "I25.700", 
                   "I25.701", "I25.702", "I25.708", "I25.709", "I25.71", "I25.710", "I25.711", "I25.712", "I25.718", 
                   "I25.719", "I25.72", "I25.720", "I25.721", "I25.722", "I25.728", "I25.729", "I25.73", "I25.730", "I25.731", 
                   "I25.732", "I25.738", "I25.739", "I25.75", "I25.750", "I25.751", "I25.752", "I25.758", "I25.759", "I25.76", 
                   "I25.760", "I25.761", "I25.762", "I25.768", "I25.769", "I25.79", "I25.790", "I25.791", "I25.792", "I25.798", 
                   "I25.799", "I25.8", "I25.81", "I25.810", "I25.811", "I25.812", "I25.82", "I25.83", "I25.84", "I25.85", "I25.89", "I25.9"]

# HD MSDRG"s dictionary
hd_msdrg_dic = {
    231: "CORONARY BYPASS WITH PTCA WITH MCC", 
    232: "CORONARY BYPASS WITH PTCA WITHOUT MCC",
    233: "CORONARY BYPASS WITH CARDIAC CATHETERIZATION WITH MCC",
    234: "CORONARY BYPASS WITH CARDIAC CATHETERIZATION WITHOUT MCC",
    235: "CORONARY BYPASS WITHOUT CARDIAC CATHETERIZATION WITH MCC",
    236: "CORONARY BYPASS WITHOUT CARDIAC CATHETERIZATION WITHOUT MCC",
    246: "PERCUTANEOUS CARDIOVASCULAR PROCEDURES WITH DRUG-ELUTING STENT WITH MCC OR 4+ ARTERIES OR STENTS",
    247: "PERCUTANEOUS CARDIOVASCULAR PROCEDURES WITH DRUG-ELUTING STENT WITHOUT MCC",
    248: "PERCUTANEOUS CARDIOVASCULAR PROCEDURES WITH NON-DRUG-ELUTING STENT WITH MCC OR 4+ ARTERIES OR STENTS",
    249: "PERCUTANEOUS CARDIOVASCULAR PROCEDURES WITH NON-DRUG-ELUTING STENT WITHOUT MCC",
    250: "PERCUTANEOUS CARDIOVASCULAR PROCEDURES WITHOUT CORONARY ARTERY STENT WITH MCC",
    251: "PERCUTANEOUS CARDIOVASCULAR PROCEDURES WITHOUT CORONARY ARTERY STENT WITHOUT MCC",
    273: "PERCUTANEOUS INTRACARDIAC PROCEDURES WITH MCC",
    274: "PERCUTANEOUS INTRACARDIAC PROCEDURES WITHOUT MCC",
    319: "OTHER ENDOVASCULAR CARDIAC VALVE PROCEDURES WITH MCC",
    320: "OTHER ENDOVASCULAR CARDIAC VALVE PROCEDURES WITHOUT MCC",
    216: "CARDIAC VALVE AND OTHER MAJOR CARDIOTHORACIC PROCEDURES WITH CARDIAC CATHETERIZATION WITH MCC",
    217: "CARDIAC VALVE AND OTHER MAJOR CARDIOTHORACIC PROCEDURES WITH CARDIAC CATHETERIZATION WITH CC",
    218: "CARDIAC VALVE AND OTHER MAJOR CARDIOTHORACIC PROCEDURES WITH CARDIAC CATHETERIZATION WITHOUT CC/MCC",
    219: "CARDIAC VALVE AND OTHER MAJOR CARDIOTHORACIC PROCEDURES WITHOUT CARDIAC CATHETERIZATION WITH MCC",
    220: "CARDIAC VALVE AND OTHER MAJOR CARDIOTHORACIC PROCEDURES WITHOUT CARDIAC CATHETERIZATION WITH CC",
    221: "CARDIAC VALVE AND OTHER MAJOR CARDIOTHORACIC PROCEDURES WITHOUT CARDIAC CATHETERIZATION WITHOUT CC/MCC",
    311: "ANGINA PECTORIS",
    302: "ATHEROSCLEROSIS WITH MCC",
    303: "ATHEROSCLEROSIS WITHOUT MCC",
    304: "HYPERTENSION WITH MCC",
    305: "HYPERTENSION WITHOUT MCC",
    299: "PERIPHERAL VASCULAR DISORDERS WITH MCC",
    300: "PERIPHERAL VASCULAR DISORDERS WITH CC",
    301: "PERIPHERAL VASCULAR DISORDERS WITHOUT CC/MCC",
    291: "HEART FAILURE AND SHOCK WITH MCC",
    292: "HEART FAILURE AND SHOCK WITH CC",
    293: "HEART FAILURE AND SHOCK WITHOUT CC/MCC",
    280: "ACUTE MYOCARDIAL INFARCTION, DISCHARGED ALIVE WITH MCC",
    281: "ACUTE MYOCARDIAL INFARCTION, DISCHARGED ALIVE WITH CC",
    282: "ACUTE MYOCARDIAL INFARCTION, DISCHARGED ALIVE WITHOUT CC/MCC",
    283: "ACUTE MYOCARDIAL INFARCTION, EXPIRED WITH MCC",
    284: "ACUTE MYOCARDIAL INFARCTION, EXPIRED WITH CC",
    285: "ACUTE MYOCARDIAL INFARCTION, EXPIRED WITHOUT CC/MCC MS"
}

# county dictionary - according to the AHCA format
county_dict = {
    1: "Alachua",
    2: "Baker",
    3: "Bay",
    4: "Bradford",
    5: "Brevard",
    6: "Broward",
    7: "Calhoun",
    8: "Charlotte",
    9: "Citrus",
    10: "Clay",
    11: "Collier",
    12: "Columbia",
    13: "Miami-Dade",
    14: "Desoto",
    15: "Dixie",
    16: "Duval",
    17: "Escambia",
    18: "Flagler",
    19: "Franklin",
    20: "Gadsden",
    21: "Gilchrist",
    22: "Glades",
    23: "Gulf",
    24: "Hamilton",
    25: "Hardee",
    26: "Hendry",
    27: "Hernando",
    28: "Highlands",
    29: "Hillsborough",
    30: "Holmes",
    31: "Indian River",
    32: "Jackson",
    33: "Jefferson",
    34: "Lafayette",
    35: "Lake",
    36: "Lee",
    37: "Leon",
    38: "Levy",
    39: "Liberty",
    40: "Madison",
    41: "Manatee",
    42: "Marion",
    43: "Martin",
    44: "Monroe",
    45: "Nassau",
    46: "Okaloosa",
    47: "Okeechobee",
    48: "Orange",
    49: "Osceola",
    50: "Palm Beach",
    51: "Pasco",
    52: "Pinellas",
    53: "Polk",
    54: "Putnam",
    55: "Saint Johns",
    56: "Saint Lucie",
    57: "Santa Rosa",
    58: "Sarasota",
    59: "Seminole",
    60: "Sumter",
    61: "Suwannee",
    62: "Taylor",
    63: "Union",
    64: "Volusia",
    65: "Wakulla",
    66: "Walton",
    67: "Washington",
    99: "Unknown"
}

chd_msdrg_list = [int(key) for key in hd_msdrg_dic]

In [ ]:
# filter patients treated for CHD in inpatient data using MScodes
chd_inp_df = inp_df[inp_df["MSDRG"].isin(chd_msdrg_list[:14])]
print(f"inpatient CHD data: {chd_inp_df.shape}")

In [ ]:
# filter patients diagnosed with CHD in outpatient data using ICD-10 CM codes
prindiag_col = ed_df[["PRINDIAG"]]
othdiag_cols = ed_df.filter(regex="^OTHDIAG\d+$")
selected_cols = pd.concat([prindiag_col, othdiag_cols], axis=1)

def check_icd_codes_in_rows(row):
    """
    Check for any of the given ICD-10 CM codes in the columns of each row
    """
    return any(code in row.values for code in chd_icd_10_list)

selected_rows = selected_cols.apply(check_icd_codes_in_rows, axis=1)
chd_ed_df = ed_df[selected_rows]

print(f"ED CHD data: {chd_ed_df.shape}")

In [ ]:
# write the final merged CHD dataframes into csv"s
chd_inp_df.to_csv("data/CHD_INPT.csv")
chd_ed_df.to_csv("data/CHD_ED.csv")

## Exploratory Data Analysis for CHD

In [ ]:
# read the inpatient and outpatient CSV"s for CHD
chd_inp_df = pd.read_csv("data/CHD_INPT.csv")
chd_ed_df = pd.read_csv("data/CHD_ED.csv")

In [ ]:
# drop unnecessary columns
inp_drop_columns = ["FACLNBR", "MCARE_NBR", "PRO_CODE", "MOD_CODE", "FAC_REGION", 
            "TYPE_SERV", "CONDTN", "EDHR_ARR", "ADM_TIME", "DIS_TIME", 
            "PAYER", "ZIPCODE", "PRINDIAG", "POA_PRIN_DIAG", "PRINPROC", 
            "WEEKDAY", "DAYSPROC", "ICUCHGS", "CCUCHGS", "PHARMCHGS", 
            "MEDCHGS", "ONCOCHGS", "LABCHGS", "RADCHGS", "OPRMCHGS", 
            "ANESCHGS", "RESPCHGS", "PHYTHCHGS", "OCCUPCHGS", "SPEECHGS", 
            "ERCHGS", "CARDIOCHGS", "TRAUMACHGS", "RECOVCHGS", "LABORCHGS", 
            "OBSERCHGS", "BEHAVCHGS", "OTHERCHGS", "TCHGS", "CERT_DATE", "ADMITDIAG", 
            "ADM_PRIOR", "PAYER_NAME", "FACL_NAME", "ROOMCHGS", "FAC_NAME", "Unnamed: 0"]

for i in range(30):
    inp_drop_columns.append(f"OTHDIAG{i+1}")
    inp_drop_columns.append(f"POA{i+1}")
    inp_drop_columns.append(f"OTHPROC{i+1}")
    inp_drop_columns.append(f"DAYS_PROC{i+1}")
for i in range(3):
    inp_drop_columns.append(f"ECMORB{i+1}")
    inp_drop_columns.append(f"POA_ECMORB{i+1}")
    inp_drop_columns.append(f"NUR{i+1}CHGS")

ed_drop_columns = ["FACLNBR", "MCARE_NBR", "PRO_CODE", "FAC_REGION", 
            "TYPE_SERV", "SERV_LOC", "FAC_REGION", "WEEKDAY", "ZIPCODE",
            "ADMSRC", "HR_ARRIVAL", "EDHR_DISCH", "PAYER", "EVALCODE1",
            "EVALCODE2", "EVALCODE3", "EVALCODE4", "EVALCODE5", "PRINDIAG", 
            "PRINPROC", "PHARMCHGS", "MEDCHGS", "LABCHGS","RADCHGS", 
            "OPRMCHGS", "ANESCHGS", "ERCHGS", "CARDIOCHGS", "TRAUMACHGS", 
            "RECOVCHGS", "OBSERCHGS", "GASTROCHGS", "LITHOCHGS", "OTHCHGS", 
            "COPD_COUNTS", "CERT_DATE", "TCHGS", "FACL_NAME", "REASON_CDE", "PAYER_NAME", "Unnamed: 0"]

for i in range(9):
    ed_drop_columns.append(f"OTHDIAG{i+1}")
for i in range(3):
    ed_drop_columns.append(f"ECMORB{i+1}")
for i in range(4):
    ed_drop_columns.append(f"OTHPROC{i+1}")
for i in range(30):
    ed_drop_columns.append(f"OTHCPT{i+1}")

# drop unnecessary inpatient columns
chd_inp_df = chd_inp_df.drop(columns=inp_drop_columns, errors="ignore")
# drop unnecessary ED columns
chd_ed_df = chd_ed_df.drop(columns=ed_drop_columns,  errors="ignore")

In [ ]:
# print column data types of CHD inpatient data
print(chd_inp_df.dtypes)

In [ ]:
# print column data types of CHD inpatient data
print(chd_ed_df.dtypes)

In [ ]:
# bucketize age groups
age_bins = [1, 34, 44, 54, 64, 74, 84, float("inf")]
age_labels = ["<35", "35-44", "45-54", "55-64", "65-74", "75-84", ">=85"]

chd_inp_df["AGE_GROUP"] = pd.cut(chd_inp_df["AGE"], bins=age_bins, labels=age_labels, include_lowest=True)
chd_ed_df["AGE_GROUP"] = pd.cut(chd_ed_df["AGE"], bins=age_bins, labels=age_labels, include_lowest=True)

chd_inp_df["AGE_GROUP"] = chd_inp_df["AGE_GROUP"].astype("object")
chd_ed_df["AGE_GROUP"] = chd_ed_df["AGE_GROUP"].astype("object")

chd_inp_df.loc[chd_inp_df["AGE"] == 0, "AGE_GROUP"] = "New Born(0-28 days)"
chd_inp_df.loc[chd_inp_df["AGE"] == 777, "AGE_GROUP"] = "Infant(29-364 days)"
chd_inp_df.loc[chd_inp_df["AGE"] == 888, "AGE_GROUP"] = ">=85"
chd_inp_df.loc[chd_inp_df["AGE"] > 100, "AGE_GROUP"] = ">=85"

chd_ed_df.loc[chd_ed_df["AGE"] == 0, "AGE_GROUP"] = "New Born(0-28 days)"
chd_ed_df.loc[chd_ed_df["AGE"] == 777, "AGE_GROUP"] = "Infant(29-364 days)"
chd_ed_df.loc[chd_ed_df["AGE"] == 888, "AGE_GROUP"] = ">=85"
chd_ed_df.loc[chd_ed_df["AGE"] > 100, "AGE_GROUP"] = ">=85"

In [ ]:
# seperate diagnosed and treated people into residents(FL) and tourists.
inp_state_filter = chd_inp_df["PTSTATE"] == "FL"
ed_state_filter = chd_ed_df["PTSTATE"] == "FL"

chd_inp_df["RESIDENT_TYPE"] = None
chd_ed_df["RESIDENT_TYPE"] = None

chd_inp_df.loc[inp_state_filter, "RESIDENT_TYPE"] = "Resident"
chd_inp_df.loc[~inp_state_filter, "RESIDENT_TYPE"] = "Non Resident"

chd_ed_df.loc[ed_state_filter, "RESIDENT_TYPE"] = "Resident"
chd_ed_df.loc[~ed_state_filter, "RESIDENT_TYPE"] = "Non Resident"

In [ ]:
# filter further to isolate CABG+PCI, CABG"s and PCI"s
chd_inp_df["PROCEDURE"] = None

chd_inp_df.loc[chd_inp_df["MSDRG"].isin(chd_msdrg_list[6:]), "PROCEDURE"] = "PCI"
chd_inp_df.loc[chd_inp_df["MSDRG"].isin(chd_msdrg_list[0:2]), "PROCEDURE"] = "CABG + PCI"
chd_inp_df.loc[chd_inp_df["MSDRG"].isin(chd_msdrg_list[2:7]), "PROCEDURE"] = "CABG"

In [ ]:
# print the count of NaN's in each column of the CHD inpatient data frame
chd_inp_df.replace(" ", np.nan,inplace=True)
print(chd_inp_df.isna().sum())

In [ ]:
# print the count of NaN's in each column of the CHD emergency data frame
chd_ed_df.replace(" ", np.nan,inplace=True)
print(chd_ed_df.isna().sum())

In [ ]:
# imputing null values in PTCOUNTY_NAME and FAC_COUNTY_NAME 

# convert fac_county into integer
inp_mask = chd_inp_df["FAC_COUNTY"].notnull()
chd_inp_df.loc[inp_mask, "FAC_COUNTY"] = chd_inp_df.loc[inp_mask, "FAC_COUNTY"].astype("int")

chd_inp_df["FAC_COUNTY_NAME"] = chd_inp_df["FAC_COUNTY"].map(county_dict)
chd_inp_df["PTCOUNTY_NAME"] = chd_inp_df["PTCOUNTY"].map(county_dict)
chd_ed_df["FAC_COUNTY_NAME"] = chd_ed_df["FAC_COUNTY"].map(county_dict)
chd_ed_df["PTCOUNTY_NAME"] = chd_ed_df["PTCOUNTY"].map(county_dict)

In [ ]:
# plot the distribution of CHD_COUNTS againts FAC_COUNTY_NAME
num_rows = len(county_dict.values()) // 3 + (len(county_dict.values()) % 3 > 0)
fig, axes = plt.subplots(num_rows, 3, figsize=(18, num_rows*4)) 
axes = axes.flatten() 

chd_counts = chd_ed_df["CHD_COUNTS"].unique()
bin_edges = np.sort(chd_counts) - 0.5

for i, county in enumerate(county_dict.values()):
    county_data = chd_ed_df[chd_ed_df["FAC_COUNTY_NAME"] == county]
    unique_chd_counts = np.sort(county_data["CHD_COUNTS"].unique())
    
    if unique_chd_counts.size > 0:
        # Define bin edges to include each unique value and an extra edge for the last bin
        bin_edges = np.append(unique_chd_counts, unique_chd_counts[-1] + 1)
        
        # Create the histogram with specified bins
        sns.histplot(data=county_data, x="CHD_COUNTS", bins=bin_edges, ax=axes[i], discrete=True)
        
        # Set title and labels for the current subplot
        axes[i].set_title(f"Distribution of CHD_COUNTS for Facility County: {county}")
        axes[i].set_xlabel("CHD_COUNTS")
        axes[i].set_ylabel("Frequency")
        axes[i].set_xticks(chd_counts)
    else:
        axes[i].set_title(f"Distribution of CHD_COUNTS for Facility County: {county}")
        
# hide any empty subplots
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
# plot the distribution of CHD_COUNTS againts PTCOUNTY_NAME
num_rows = len(county_dict.values()) // 3 + (len(county_dict.values()) % 3 > 0)
fig, axes = plt.subplots(num_rows, 3, figsize=(18, num_rows*4)) 
axes = axes.flatten() 

chd_counts = chd_ed_df["CHD_COUNTS"].unique()
bin_edges = np.sort(chd_counts) - 0.5

for i, county in enumerate(county_dict.values()):
    county_data = chd_ed_df[chd_ed_df["PTCOUNTY_NAME"] == county]
    unique_chd_counts = np.sort(county_data["CHD_COUNTS"].unique())
    
    if unique_chd_counts.size > 0:
        # Define bin edges to include each unique value and an extra edge for the last bin
        bin_edges = np.append(unique_chd_counts, unique_chd_counts[-1] + 1)
        
        # Create the histogram with specified bins
        sns.histplot(data=county_data, x="CHD_COUNTS", bins=bin_edges, ax=axes[i], discrete=True)
        
        # Set title and labels for the current subplot
        axes[i].set_title(f"Distribution of CHD_COUNTS for Patient County: {county}")
        axes[i].set_xlabel("CHD_COUNTS")
        axes[i].set_ylabel("Frequency")
        axes[i].set_xticks(chd_counts)
    else:
        axes[i].set_title(f"Distribution of CHD_COUNTS for Patient County: {county}")
        
# hide any empty subplots
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
# get the patient counties and years where the facility county is missing
chd_inp_missing_pt_counties = chd_inp_df.loc[chd_inp_df["FAC_COUNTY"].isna(), "PTCOUNTY"].unique()
chd_inp_years = chd_inp_df.loc[chd_inp_df["FAC_COUNTY"].isna(), "YEAR"].unique()
resident_type = chd_inp_df.loc[chd_inp_df["FAC_COUNTY"].isna(), "RESIDENT_TYPE"].unique()

In [ ]:
grouped = chd_inp_df.groupby(["YEAR", "PTCOUNTY_NAME", "FAC_COUNTY_NAME", "RESIDENT_TYPE"]).size().reset_index(name="PROC_COUNT")

In [ ]:
# plot the percentage of other patients from the same patient county that get treated in different facility counties
max_percentage_dict = {}

for county in chd_inp_missing_pt_counties:
    records = chd_inp_df.loc[(chd_inp_df["PTCOUNTY"] == county) & (chd_inp_df["RESIDENT_TYPE"] == "Resident")]
    if not records.empty:
        missing = len(records[records["FAC_COUNTY"].isna()])
        records["FAC_COUNTY"] = records["FAC_COUNTY"] .astype(str)
        plt.figure(figsize=(10, 6))  # You can adjust the size as needed
        # Create a countplot with percentages
        ax = sns.countplot(data=records, x="FAC_COUNTY", order=records["FAC_COUNTY"].value_counts().index)
        
        # Calculate the percentage for each bar
        total = len(records)  # Total number of records for the county
        for p in ax.patches:
            percentage = f'{100 * p.get_height() / total:.1f}%'
            x = p.get_x() + p.get_width() / 2
            y = p.get_height()
            ax.annotate(percentage, (x, y), ha='center', va='center')

        value_counts = records["FAC_COUNTY"].value_counts(normalize=True)
        percentages = value_counts * 100
        max_percentage = percentages.max()
        max_percentage_fac_county = percentages.idxmax()

        # Save the max percentage and corresponding FAC_COUNTY to the dictionary
        max_percentage_dict[county] = {'FAC_COUNTY': int(max_percentage_fac_county), 'Percentage': max_percentage}
        
        # Set the title and labels
        plt.title(f"Percentage of patients from {county} who got treated in different counties and missing:{missing}")
        plt.xticks(rotation=90)  # Rotate the x-axis labels for better readability
        plt.xlabel("Facility County")
        plt.ylabel("Percentage")
        
        plt.tight_layout()  # Adjust layout to prevent clipping of ylabel
        plt.show()  

By looking at the percentages of those patients getting treated from different facility counties for each patient county shown above, we can observe that most of the times a single county received a large volume of patients for treatment. Therefore for the missing facility counties w can impute with the facility county with max percentage from other patients from same patient county.

In [ ]:
# imputing missing facility counties based on the argument mentioned above.
for county in chd_inp_missing_pt_counties:
    if county != 99:
        chd_inp_df.loc[(chd_inp_df["PTCOUNTY"] == county) & (chd_inp_df["FAC_COUNTY"].isna()), "FAC_COUNTY"] = max_percentage_dict[county]["FAC_COUNTY"]
    else:
        chd_inp_df.loc[(chd_inp_df["PTCOUNTY"] == 99) & (chd_inp_df["FAC_COUNTY"].isna()), "FAC_COUNTY"] = 16

In [ ]:
# replace the missing facility county names from the county dictionary.
chd_inp_df["FAC_COUNTY_NAME"] = chd_inp_df["FAC_COUNTY"].map(county_dict)

In [ ]:
# print the count of NaN's in each column of the CHD inpatient data frame
chd_inp_df.isna().sum()

In [ ]:
# plot a time series plot to see count of treatment procedures conducted in each county by quater per year
chd_inp_df["YEAR_QTR"] = chd_inp_df["YEAR"].astype(str) + "QTR" + chd_inp_df["QTR"].astype(str)
chd_inp_proc_agg_by_fac = chd_inp_df.groupby(["QTR", "FAC_COUNTY_NAME", "PROCEDURE", "RESIDENT_TYPE"]).size().reset_index(name="PROC_COUNT")

treatments = ["PCI", "CABG", "CABG + PCI"]
palette = sns.color_palette("husl", len(treatments))
treatment_colors = dict(zip(treatments, palette))

for county in county_dict.values():
    county_data = chd_inp_proc_agg_by_fac[((chd_inp_proc_agg_by_fac["FAC_COUNTY_NAME"] == county) & (chd_inp_proc_agg_by_fac["RESIDENT_TYPE"] == "Resident"))]
    if county_data.size > 0:
        plt.figure(figsize=(10, 6))
        sns.lineplot(data=county_data, x="YEAR", y="PROC_COUNT", hue="PROCEDURE", marker="o", palette=treatment_colors)

        plt.title(f"Procedure counts by year & qtr for {county}")
        plt.xlabel("year_qtr")
        plt.ylabel("count")
        plt.xticks(rotation=45)
        plt.legend(title="Procedure", bbox_to_anchor=(1.05, 1), loc="upper left")

        plt.tight_layout()
        plt.show()
    else:
        print(f"No data available for {county}")

In [ ]:
# read the population by county csv file
usecols = ["County"]
for i in range(2015, 2024):
    usecols.append(str(i))

pop_by_county = pd.read_csv("data/PopByCounty.csv", usecols=usecols)

pop_by_county_dics = {}
for i, row in pop_by_county.iterrows():
    county = row["County"]
    county_data = {year: row[year] for year in usecols if year != "County"}
    pop_by_county_dics[county] = county_data

In [ ]:
# melt the dataframe
pop_by_county_melt = melted_df = pd.melt(pop_by_county, id_vars=['County'], var_name='Year', value_name='Population')
pop_by_county_melt

In [ ]:
# remove newborn records - since this is an exceptional scenario, we are ignoring this from our study for simplicity
chd_inp_special_df = chd_inp_df[chd_inp_df["AGE_GROUP"].isin(["New Born(0-28 days)", "<35"])]
chd_ed_special_df = chd_ed_df[chd_ed_df["AGE_GROUP"].isin(["New Born(0-28 days)", "<35"])]
chd_inp_df = chd_inp_df[~(chd_inp_df["AGE_GROUP"].isin(["New Born(0-28 days)", "<35"]))]
chd_ed_df = chd_ed_df[~(chd_ed_df["AGE_GROUP"].isin(["New Born(0-28 days)", "<35"]))]

In [ ]:
# create new column concatanating YEAR and QTR
chd_ed_df["YEAR_QTR"] = chd_ed_df["YEAR"].astype(str) + "QTR" + chd_ed_df["QTR"].astype(str)

In [ ]:
# plot a time series plot to show the county of patients diagnosed in counties by year.
chd_inp_proc_agg_by_pt = chd_inp_df.groupby(["YEAR", "PTCOUNTY_NAME", "PROCEDURE", "RESIDENT_TYPE"]).size().reset_index(name="PROC_COUNT")
chd_ed_diag_agg_by_pt = chd_ed_df.groupby(["YEAR", "PTCOUNTY_NAME", "RESIDENT_TYPE"]).size().reset_index(name="DIAG_COUNT")

for county in county_dict.values():
    fig, axes = plt.subplots(3, 1, figsize=(20, 15)) 

    county_diag_data = chd_ed_diag_agg_by_pt[(chd_ed_diag_agg_by_pt["PTCOUNTY_NAME"] == county) & (chd_ed_diag_agg_by_pt["RESIDENT_TYPE"] == "Resident")]
    county_pop = pop_by_county_melt[pop_by_county_melt["County"] == county]

    sns.lineplot(data=county_pop, x="Year", y="Population", ax=axes[0], marker="o")
    axes[0].set_title(f"Population in {county}")
    axes[0].set_xlabel("Year")
    axes[0].set_ylabel("Diagnosis Count")
    
    if county_diag_data.size > 0:    
        sns.lineplot(data=county_diag_data, x="YEAR", y="DIAG_COUNT", ax=axes[1], marker="o")
        axes[1].set_title(f"Diagnosis Counts for {county}")
        axes[1].set_xlabel("Year")
        axes[1].set_ylabel("Diagnosis Count")

        county_proc_data = chd_inp_proc_agg_by_pt[(chd_inp_proc_agg_by_pt["PTCOUNTY_NAME"] == county) & (chd_inp_proc_agg_by_pt["RESIDENT_TYPE"] == "Resident")]
        sns.lineplot(data=county_proc_data, x="YEAR", y="PROC_COUNT", hue="PROCEDURE", ax=axes[2], palette=treatment_colors, marker="o")
        axes[2].set_title(f"Procedure Counts for {county}")
        axes[2].set_xlabel("Year")
        axes[2].set_ylabel("Procedure Count")

        plt.suptitle(f"Counts for {county}", y=1.05)
        plt.tight_layout(pad=3.0)
        plt.show()

In [ ]:
# aggregate the original dataframes.
chd_inp_proc_agg_by_pt = chd_inp_df.groupby(["YEAR", "PTCOUNTY_NAME", "PROCEDURE", "RESIDENT_TYPE"]).size().reset_index(name="PROC_COUNT")
chd_ed_diag_agg_by_pt = chd_ed_df.groupby(["YEAR", "PTCOUNTY_NAME", "RESIDENT_TYPE"]).size().reset_index(name="DIAG_COUNT")

In [ ]:
# create two new columns POPULATION, PROC_FRAC_BY_COUNTY = PROC_COUNT/POPULATION and DIAG_FRAC_BY_COUNTY = DIAG_COUNT/POPULATION
for county in pop_by_county_dics.keys():
    for year in range(2015, 2024):
        # Create the correct mask for current county and year
        county_proc_mask = (chd_inp_proc_agg_by_pt["PTCOUNTY_NAME"] == county) & (chd_inp_proc_agg_by_pt["YEAR"] == year)
        county_diag_mask = (chd_ed_diag_agg_by_pt["PTCOUNTY_NAME"] == county) & (chd_ed_diag_agg_by_pt["YEAR"] == year)
        # Get the population for the current county and year
        population = pop_by_county_dics[county].get(str(year))
        # Apply the calculation if population is not None
        if population is not None:
            chd_inp_proc_agg_by_pt.loc[county_proc_mask, "POPULATION"] = population
            chd_inp_proc_agg_by_pt.loc[county_proc_mask, "PROC_FRAC_BY_COUNTY"] = (chd_inp_proc_agg_by_pt.loc[county_proc_mask, "PROC_COUNT"] / population)
            chd_ed_diag_agg_by_pt.loc[county_diag_mask, "POPULATION"] = population
            chd_ed_diag_agg_by_pt.loc[county_diag_mask, "DIAG_FRAC_BY_COUNTY"] = (chd_ed_diag_agg_by_pt.loc[county_diag_mask, "DIAG_COUNT"] / population)

In [ ]:
# Get unique counties
unique_counties = chd_inp_proc_agg_by_pt["PTCOUNTY_NAME"].unique()

# plot the distribution of the PROC_FRAC_BY_COUNTY
num_plots_per_row = 3
total_plots = len(unique_counties) * len(treatments[:-1])
num_rows = total_plots // num_plots_per_row + (total_plots % num_plots_per_row > 0)
fig, axes = plt.subplots(num_rows, num_plots_per_row, figsize=(20, num_rows * 5))
axes = axes.flatten()  

plot_index = 0 
for county in unique_counties:
    for procedure in treatments[:-1]:
        county_proc_data =  chd_inp_proc_agg_by_pt[(chd_inp_proc_agg_by_pt["PTCOUNTY_NAME"] == county) & 
                                                   (chd_inp_proc_agg_by_pt["RESIDENT_TYPE"] == "Resident") & 
                                                   (chd_inp_proc_agg_by_pt["PROCEDURE"] == procedure)]
        sns.histplot(county_proc_data["PROC_FRAC_BY_COUNTY"].dropna(), ax=axes[plot_index], kde=True)
        axes[plot_index].set_title(f"{procedure} in {county}")
        axes[plot_index].set_xlabel("PROC_FRAC_BY_COUNTY")
        axes[plot_index].set_ylabel("Frequency")
        
        plot_index += 1 
for j in range(plot_index, len(axes)):
    fig.delaxes(axes[j])

plt.show()

In [ ]:
# Get unique counties
unique_counties = chd_inp_proc_agg_by_pt["PTCOUNTY_NAME"].unique()

# plot a scatter plot with regression line for PROC_FRAC_BY_COUNTY againts YEAR
num_plots_per_row = 4
total_plots = len(unique_counties) * len(treatments[:-1])
num_rows = total_plots // num_plots_per_row + (total_plots % num_plots_per_row > 0)
fig, axes = plt.subplots(num_rows, num_plots_per_row, figsize=(20, num_rows * 5))
axes = axes.flatten()

plot_index = 0 
for county in unique_counties:
    for procedure in treatments[:-1]:
        county_proc_data = chd_inp_proc_agg_by_pt[(chd_inp_proc_agg_by_pt["PTCOUNTY_NAME"] == county) & 
                                                   (chd_inp_proc_agg_by_pt["RESIDENT_TYPE"] == "Resident") & 
                                                   (chd_inp_proc_agg_by_pt["PROCEDURE"] == procedure)]
        
        sns.regplot(data=county_proc_data, x="YEAR", y="PROC_FRAC_BY_COUNTY", logx=True, ax=axes[plot_index])
        axes[plot_index].set_title(f"{procedure} in {county}")
        axes[plot_index].set_xlabel("YEAR")
        axes[plot_index].set_ylabel("PROC_FRAC_BY_COUNTY")
        
        plot_index += 1 
for j in range(plot_index, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
# add the count of CABG + PCI into both CABG & and PCI and remove CABG + PCI. This is due to the sparse data points for CABG + PCI since its a rare case.
for county in chd_inp_proc_agg_by_pt['PTCOUNTY_NAME'].unique():
    for year in chd_inp_proc_agg_by_pt['YEAR'].unique():
        chd_inp_proc_agg_by_pt["PROC_FRAC_BY_COUNTY"] =  chd_inp_proc_agg_by_pt["PROC_FRAC_BY_COUNTY"].fillna(0)
        chd_inp_proc_agg_by_pt.loc[(chd_inp_proc_agg_by_pt["PROCEDURE"] == "CABG")
                                    & (chd_inp_proc_agg_by_pt["PTCOUNTY_NAME"] == county)
                                    & (chd_inp_proc_agg_by_pt["YEAR"] == year), "PROC_FRAC_BY_COUNTY"] = sum(chd_inp_proc_agg_by_pt.loc[(chd_inp_proc_agg_by_pt["PROCEDURE"] == "CABG + PCI")
                                                                                                                                                    & (chd_inp_proc_agg_by_pt["PTCOUNTY_NAME"] == county)
                                                                                                                                                    & (chd_inp_proc_agg_by_pt["YEAR"] == year), "PROC_FRAC_BY_COUNTY"],
                                                                                                                        chd_inp_proc_agg_by_pt.loc[(chd_inp_proc_agg_by_pt["PROCEDURE"] == "CABG")
                                                                                                                                                    & (chd_inp_proc_agg_by_pt["PTCOUNTY_NAME"] == county)
                                                                                                                                                    & (chd_inp_proc_agg_by_pt["YEAR"] == year), "PROC_FRAC_BY_COUNTY"])
        chd_inp_proc_agg_by_pt.loc[(chd_inp_proc_agg_by_pt["PROCEDURE"] == "PCI")
                                    & (chd_inp_proc_agg_by_pt["PTCOUNTY_NAME"] == county)
                                    & (chd_inp_proc_agg_by_pt["YEAR"] == year), "PROC_FRAC_BY_COUNTY"] = sum(chd_inp_proc_agg_by_pt.loc[(chd_inp_proc_agg_by_pt["PROCEDURE"] == "CABG + PCI")
                                                                                                                                                    & (chd_inp_proc_agg_by_pt["PTCOUNTY_NAME"] == county)
                                                                                                                                                    & (chd_inp_proc_agg_by_pt["YEAR"] == year), "PROC_FRAC_BY_COUNTY"],
                                                                                                                        chd_inp_proc_agg_by_pt.loc[(chd_inp_proc_agg_by_pt["PROCEDURE"] == "PCI")
                                                                                                                                                    & (chd_inp_proc_agg_by_pt["PTCOUNTY_NAME"] == county)
                                                                                                                                                    & (chd_inp_proc_agg_by_pt["YEAR"] == year), "PROC_FRAC_BY_COUNTY"])
chd_inp_proc_agg_by_pt = chd_inp_proc_agg_by_pt[~((chd_inp_proc_agg_by_pt["PROCEDURE"] == "CABG + PCI"))]

<b> Using a GLM we are going to find the best fit line for the time series data. The coefficients will be used in a clustering algorithm to cluster the counties that have similar trends for diagnostics and treatments procedures seperately <b/> 
1. First normalize the countys by dividng the count by the population(i.e. PROC_FRAC_BY_COUNTY and DIAG_FRAC_BY_COUNTY)
2. Year would be the independent variables. Encode as a continous value between 0.1 and 0.99.

In [ ]:
# fit a GLM for procedure fraction and diagnostic fractions to estimate the trend.
num_plots_per_row = 3
total_plots = len(unique_counties) * (len(treatments[:-1]) + 1)
num_rows = total_plots // num_plots_per_row + (total_plots % num_plots_per_row > 0)
fig, axes = plt.subplots(num_rows, num_plots_per_row, figsize=(20, num_rows * 5))
axes = axes.flatten()  

year_to_x = {year: x for year, x in zip(chd_inp_proc_agg_by_pt["YEAR"].unique(), np.linspace(0.1, 0.99, len(chd_inp_proc_agg_by_pt["YEAR"].unique())))}
diag_coeff_vector = {}
proc_coeff_vector = {}

plot_index = 0
for county in unique_counties:
    proc_coeff_vector[county] = []
    diag_coeff_vector[county] = []
    for procedure in treatments[:-1]:
        # Filter data for the current county and procedure
        county_proc_data = chd_inp_proc_agg_by_pt[(chd_inp_proc_agg_by_pt['PTCOUNTY_NAME'] == county) &
                                                   (chd_inp_proc_agg_by_pt["RESIDENT_TYPE"] == "Resident") & 
                                                   (chd_inp_proc_agg_by_pt['PROCEDURE'] == procedure)]
        
        # Drop rows with NaN values in either 'PROC_FAC_BY_COUNTY' or 'POPULATION'
        county_proc_data = county_proc_data.dropna(subset=['PTCOUNTY_NAME', 'PROC_FRAC_BY_COUNTY'])
        if not county_proc_data.empty:
            # fit glm to treatment procedures
            county_proc_data['X'] = county_proc_data['YEAR'].map(year_to_x)
            model = smf.glm(formula="PROC_FRAC_BY_COUNTY ~ X ", data=county_proc_data, 
                            family=sm.families.Binomial()).fit()
            proc_coeff_vector[county].append(model.params["Intercept"])
            proc_coeff_vector[county].append(model.params["X"])

            # plot the fitted GLM line in the scatter plot.
            x_pred = np.linspace(county_proc_data["X"].min(), county_proc_data["X"].max(), 100)
            y_pred = model.predict(pd.DataFrame({"X": x_pred}))
            axes[plot_index].scatter(county_proc_data["X"], county_proc_data["PROC_FRAC_BY_COUNTY"], label="Actual", alpha=0.6)
            axes[plot_index].plot(x_pred, y_pred, color='red', label='Fitted GLM')
            axes[plot_index].set_title(f"{procedure} in {county}")
            axes[plot_index].set_xlabel("Year")
            axes[plot_index].set_ylabel("PROC_FRAC_BY_COUNTY")
            axes[plot_index].legend()

            plot_index += 1

    county_diag_data = chd_ed_diag_agg_by_pt[(chd_ed_diag_agg_by_pt['PTCOUNTY_NAME'] == county) &
                                                   (chd_ed_diag_agg_by_pt["RESIDENT_TYPE"] == "Resident")]
    # fit glm to diagnostics
    if not county_diag_data.empty:
        county_diag_data['X'] = county_diag_data['YEAR'].map(year_to_x)
        model = smf.glm(formula="DIAG_FRAC_BY_COUNTY ~ X ", data=county_diag_data, 
                        family=sm.families.Binomial()).fit()
        diag_coeff_vector[county].append(model.params["Intercept"])
        diag_coeff_vector[county].append(model.params["X"])

        # plot the fitted GLM line in the scatter plot.
        x_pred = np.linspace(county_diag_data["X"].min(), county_diag_data["X"].max(), 100)
        y_pred = model.predict(pd.DataFrame({"X": x_pred}))
        axes[plot_index].scatter(county_diag_data["X"], county_diag_data["DIAG_FRAC_BY_COUNTY"], label="Actual", alpha=0.6)
        axes[plot_index].plot(x_pred, y_pred, color='red', label='Fitted GLM')
        axes[plot_index].set_title(f"diagnosis in {county}")
        axes[plot_index].set_xlabel("Year")
        axes[plot_index].set_ylabel("DIAG_FRAC_BY_COUNTY")
        axes[plot_index].legend()

        plot_index += 1

plt.tight_layout()
plt.show()

In [ ]:
# find the maximum length of coefficient lists
max_length = max(len(coeffs) for coeffs in proc_coeff_vector.values() if coeffs)

# pad coefficient lists with zeros to ensure uniform length
padded_proc_coeffs = {county: coeffs + [0] * (max_length - len(coeffs)) for county, coeffs in proc_coeff_vector.items()}

# convert the dictionary to a DataFrame
proc_coeff_df = pd.DataFrame.from_dict(padded_proc_coeffs, orient="index")

In [ ]:
# find the maximum length of coefficient lists
max_length = max(len(coeffs) for coeffs in diag_coeff_vector.values() if coeffs)

# pad coefficient lists with zeros to ensure uniform length
padded_diag_coeffs = {county: coeffs + [0] * (max_length - len(coeffs)) for county, coeffs in diag_coeff_vector.items()}

# convert the dictionary to a DataFrame
diag_coeff_df = pd.DataFrame.from_dict(padded_diag_coeffs, orient="index")

In [ ]:
# cluster the treatment procedure GLM coefficients using K-Means.
inertias = []
K = range(1, 10) 
for k in K:
    kmeans_model = KMeans(n_clusters=k, random_state=42)
    kmeans_model.fit(proc_coeff_df)
    inertias.append(kmeans_model.inertia_)

plt.figure(figsize=(8, 4))
plt.plot(K, inertias, "bx-")
plt.xlabel("Number of county clusters in procedures")
plt.ylabel("Sum of squared distances (Inertia)")
plt.title("The Elbow Method showing the optimal k")
plt.xticks(K)
plt.show()

In [ ]:
# cluster the treatment procedure GLM coefficients using K-Means
inertias = []
K = range(1, 10) 
for k in K:
    kmeans_model = KMeans(n_clusters=k, random_state=42)
    kmeans_model.fit(diag_coeff_df)
    inertias.append(kmeans_model.inertia_)

plt.figure(figsize=(8, 4))
plt.plot(K, inertias, "bx-")
plt.xlabel("Number of county clusters in diagnosis")
plt.ylabel("Sum of squared distances (Inertia)")
plt.title("The Elbow Method showing the optimal k")
plt.xticks(K)
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(proc_coeff_df)
proc_coeff_df["cluster"] = kmeans.labels_

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(diag_coeff_df)
diag_coeff_df["cluster"] = kmeans.labels_

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 5)) 

# let's say we use the first two coefficients (columns 0 and 1) for the x and y axes
sns.scatterplot(data=proc_coeff_df, x=0, y=1, hue='cluster', palette='viridis', ax=axes[0])
sns.scatterplot(data=proc_coeff_df, x=2, y=3, hue='cluster', palette='viridis', ax=axes[1])

plt.legend(title='Cluster')

plt.show()

In [ ]:
# let's say we use the first two coefficients (columns 0 and 1) for the x and y axes
sns.scatterplot(data=diag_coeff_df, x=0, y=1, hue='cluster', palette='viridis')
plt.legend(title='Cluster')
plt.show()

In [ ]:
print(proc_coeff_df[proc_coeff_df["cluster"] == 0].index)
print(diag_coeff_df[diag_coeff_df["cluster"] == 0].index)

In [ ]:
print(proc_coeff_df[proc_coeff_df["cluster"] == 1].index)
print(diag_coeff_df[diag_coeff_df["cluster"] == 1].index)
print(diag_coeff_df[diag_coeff_df["cluster"] == 2].index)

NOTE: Hierachical clustering was used as the best clustering technique for this use case. K-Means clustering does provid a way see how coefficients of PCI and CABG cluster the data.

In [ ]:
# cluster the diagnostic GLM coefficients using Agglomerative Hierarchical clustering.
clustering = AgglomerativeClustering(n_clusters=5, linkage="ward")
clustering.fit(diag_coeff_df)

In [ ]:
# plot dendogram
diag_linkage_matrix = ward(diag_coeff_df)
plt.figure(figsize=(10, 7))
dendrogram(diag_linkage_matrix, labels=diag_coeff_df.index.tolist(), leaf_rotation=90, leaf_font_size=10)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Sample index')
plt.ylabel('Distance')
plt.show()

In [ ]:
clusters = fcluster(diag_linkage_matrix, 5, criterion="distance")
diag_coeff_df["cluster"] = clusters

In [ ]:
# cluster the treatment procedures GLM coefficients using Agglomerative Hierarchical clustering.
clustering = AgglomerativeClustering(n_clusters=5, linkage="ward")
clustering.fit(proc_coeff_df.loc[:,2:3])
proc_linkage_matrix = ward(proc_coeff_df.loc[:,2:3])
plt.figure(figsize=(10, 7))
dendrogram(proc_linkage_matrix, labels=proc_coeff_df.index.tolist(), leaf_rotation=90, leaf_font_size=10)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Sample index')
plt.ylabel('Distance')
plt.show()

In [ ]:
clusters = fcluster(proc_linkage_matrix, 5, criterion="distance")
proc_coeff_df["cluster"] = clusters

In [ ]:
chd_inp_df["CLUSTER"] = np.nan
chd_inp_proc_agg_by_pt["CLUSTER"] = np.nan

for x, row in proc_coeff_df.iterrows():
    chd_inp_df.loc[chd_inp_df["PTCOUNTY_NAME"] == x, "CLUSTER"] = str(row["cluster"])
    chd_inp_proc_agg_by_pt.loc[chd_inp_proc_agg_by_pt["PTCOUNTY_NAME"] == x,  "CLUSTER"] = int(row["cluster"])

chd_ed_df["CLUSTER"] = np.nan
chd_ed_diag_agg_by_pt["CLUSTER"] = np.nan

for x, row in diag_coeff_df.iterrows():
    chd_ed_df.loc[chd_ed_df["PTCOUNTY_NAME"] == x, "CLUSTER"] = str(row["cluster"])
    chd_ed_diag_agg_by_pt.loc[chd_ed_diag_agg_by_pt["PTCOUNTY_NAME"] == x,  "CLUSTER"] = int(row["cluster"])

In [ ]:
pop_by_county_melt["Year"] = pop_by_county_melt["Year"].astype("int")

In [ ]:
# plot time series plots for the clustered counties to see how they differ
chd_inp_proc_clus_pt = chd_inp_df.groupby(["YEAR", "PTCOUNTY_NAME", "PROCEDURE", "RESIDENT_TYPE", "CLUSTER"]).size().reset_index(name="PROC_COUNT")
chd_ed_diag_clus_pt = chd_ed_df.groupby(["YEAR", "PTCOUNTY_NAME", "RESIDENT_TYPE", "CLUSTER"]).size().reset_index(name="DIAG_COUNT")

df_inp = chd_inp_proc_clus_pt.rename(columns={"PTCOUNTY_NAME": "County", "YEAR": "Year"})
df_ed = chd_ed_diag_clus_pt.rename(columns={"PTCOUNTY_NAME": "County", "YEAR": "Year"})

merged_inp_df = pd.merge(df_inp, pop_by_county_melt, how="left", on=["Year", "County"])
merged_inp_df['FRAC_PROC_BY_COUNTY'] = merged_inp_df['PROC_COUNT'] / merged_inp_df['Population']
merged_ed_df = pd.merge(df_ed, pop_by_county_melt, how="left", on=["Year", "County"])
merged_ed_df['FRAC_DIAG_BY_COUNTY'] = merged_ed_df['DIAG_COUNT'] / merged_ed_df['Population']

for cluster in ("1.0", "2.0", "3.0"):
    fig, axes = plt.subplots(2, 1, figsize=(20, 15)) 

    county_diag_data = merged_ed_df[(merged_ed_df["CLUSTER"] == cluster)]
    sns.lineplot(data=county_diag_data, x="Year", y="FRAC_DIAG_BY_COUNTY", ax=axes[0], marker="o")
    axes[0].set_title(f"Diagnosis Counts for patient county {cluster}")
    axes[0].set_xlabel("Year")
    axes[0].set_ylabel("Diagnosis Count")

    county_proc_data = merged_inp_df[(merged_inp_df["CLUSTER"] == cluster)]
    sns.lineplot(data=county_proc_data, x="Year", y="FRAC_PROC_BY_COUNTY", hue="PROCEDURE", ax=axes[1], palette=treatment_colors, marker="o")
    axes[1].set_title(f"Procedure Counts for patient county {cluster}")
    axes[1].set_xlabel("Year")
    axes[1].set_ylabel("Procedure Count")
    
    plt.tight_layout(pad=3.0)
    plt.show()

In [ ]:
# group the original datasets by year quater this time.
chd_inp_proc_pt_by_age = chd_inp_df.groupby(["YEAR", "YEAR_QTR", "AGE_GROUP","PTCOUNTY_NAME", "PROCEDURE", "RESIDENT_TYPE"]).size().reset_index(name="PROC_COUNT")
chd_ed_diag_pt_by_age = chd_ed_df.groupby(["YEAR", "YEAR_QTR", "AGE_GROUP", "PTCOUNTY_NAME", "RESIDENT_TYPE"]).size().reset_index(name="DIAG_COUNT")

In [ ]:
# add the countis of PCI + CABG into PCI and CABG and remove PCI + CABG
for county in chd_inp_proc_pt_by_age["PTCOUNTY_NAME"].unique():
    for year in chd_inp_proc_pt_by_age["YEAR_QTR"].unique():
        for age_group in chd_inp_proc_pt_by_age["AGE_GROUP"].unique():
            chd_inp_proc_pt_by_age["PROC_COUNT"] =  chd_inp_proc_pt_by_age["PROC_COUNT"].fillna(0)
            chd_inp_proc_pt_by_age.loc[(chd_inp_proc_pt_by_age["PROCEDURE"] == "CABG")
                                        & (chd_inp_proc_pt_by_age["PTCOUNTY_NAME"] == county)
                                        & (chd_inp_proc_pt_by_age["YEAR_QTR"] == year)
                                        & (chd_inp_proc_pt_by_age["AGE_GROUP"] == age_group), "PROC_COUNT"] = sum(chd_inp_proc_pt_by_age.loc[(chd_inp_proc_pt_by_age["PROCEDURE"] == "CABG + PCI")
                                                                                                                                                        & (chd_inp_proc_pt_by_age["PTCOUNTY_NAME"] == county)
                                                                                                                                                        & (chd_inp_proc_pt_by_age["YEAR_QTR"] == year)
                                                                                                                                                        & (chd_inp_proc_pt_by_age["AGE_GROUP"] == age_group), "PROC_COUNT"],
                                                                                                                            chd_inp_proc_pt_by_age.loc[(chd_inp_proc_pt_by_age["PROCEDURE"] == "CABG")
                                                                                                                                                        & (chd_inp_proc_pt_by_age["PTCOUNTY_NAME"] == county)
                                                                                                                                                        & (chd_inp_proc_pt_by_age["YEAR_QTR"] == year)
                                                                                                                                                        & (chd_inp_proc_pt_by_age["AGE_GROUP"] == age_group), "PROC_COUNT"])
            chd_inp_proc_pt_by_age.loc[(chd_inp_proc_pt_by_age["PROCEDURE"] == "PCI")
                                        & (chd_inp_proc_pt_by_age["PTCOUNTY_NAME"] == county)
                                        & (chd_inp_proc_pt_by_age["YEAR_QTR"] == year)
                                        & (chd_inp_proc_pt_by_age["AGE_GROUP"] == age_group), "PROC_COUNT"] = sum(chd_inp_proc_pt_by_age.loc[(chd_inp_proc_pt_by_age["PROCEDURE"] == "CABG + PCI")
                                                                                                                                                        & (chd_inp_proc_pt_by_age["PTCOUNTY_NAME"] == county)
                                                                                                                                                        & (chd_inp_proc_pt_by_age["YEAR_QTR"] == year)
                                                                                                                                                        & (chd_inp_proc_pt_by_age["AGE_GROUP"] == age_group), "PROC_COUNT"],
                                                                                                                            chd_inp_proc_pt_by_age.loc[(chd_inp_proc_pt_by_age["PROCEDURE"] == "PCI")
                                                                                                                                                        & (chd_inp_proc_pt_by_age["PTCOUNTY_NAME"] == county)
                                                                                                                                                        & (chd_inp_proc_pt_by_age["YEAR_QTR"] == year)
                                                                                                                                                        & (chd_inp_proc_pt_by_age["AGE_GROUP"] == age_group), "PROC_COUNT"])
chd_inp_proc_pt_by_age = chd_inp_proc_pt_by_age[~((chd_inp_proc_pt_by_age["PROCEDURE"] == "CABG + PCI"))]

In [ ]:
def get_population_by_age_group(file_name):
  """
  Group the population by age group and create seperate dataframes.
  """
  pop_list = {}
  # open the excel file
  with pd.ExcelFile(file_name) as xls:
    # read through age group sheets
    for i in range(35, 85, 10):
      df_males = pd.read_excel(xls, sheet_name=f'{i}to{i+9}-Males')
      df_females = pd.read_excel(xls, sheet_name=f'{i}to{i+9}-Females')
      # aggregate males and females by age groups
      pop_list[f'{i}-{i+9}'] = aggregate_gender(df_males, df_females)

    df_males = pd.read_excel(xls, sheet_name=f'85+-Males')
    df_females = pd.read_excel(xls, sheet_name=f'85+-Females')
    # aggregate males and females by age groups
    pop_list['>=85'] = aggregate_gender(df_males, df_females)
  return pop_list

def aggregate_gender(df_males, df_females):
  """
  Aggregate males and females population count.
  """
  # combine the two data frames
  df_total = pd.merge(df_males, df_females, on="County", suffixes=("_male", "_female"))
  for year in range(1970, 2024):  # adjust range according to your data
    year_ = str(year)
    if year_+'_male' in df_total and year_+'_female' in df_total:
        df_total[year] = df_total[year_+'_male'] + df_total[year_+'_female']
        df_total.drop(columns=[year_+'_male', year_+'_female'], inplace=True)
  return df_total

In [ ]:
# prepare population dataframe
pop_list_by_age_county = get_population_by_age_group("data/PopByCountyGroupsConfigured.xlsx")

melted_pop_dfs = []
for age_group, df in pop_list_by_age_county.items():
    melted_df = df.melt(id_vars="County", var_name="Year", value_name=age_group)
    melted_pop_dfs.append(melted_df)
pop_df = melted_pop_dfs[0]
for df in melted_pop_dfs[1:]:
    pop_df = pd.merge(pop_df, df, on=["County", "Year"], how="inner")
numeric_columns = pop_df.select_dtypes(include=['float']).columns
pop_df[numeric_columns] = pop_df[numeric_columns].apply(lambda x: x.astype(int))

In [ ]:
# calculate the PROC_FRAC_BY_COUNTY and DIAG_FRAC_BY_COUNTY
for county in pop_df["County"].unique():
    for year in range(2015, 2024):
        for age_group in chd_inp_proc_pt_by_age["AGE_GROUP"].unique():
            # create the mask for current county and year
            county_proc_mask = (chd_inp_proc_pt_by_age["PTCOUNTY_NAME"] == county) & (chd_inp_proc_pt_by_age["YEAR"] == year) & (chd_inp_proc_pt_by_age["AGE_GROUP"] == age_group)
            county_diag_mask = (chd_ed_diag_pt_by_age["PTCOUNTY_NAME"] == county) & (chd_ed_diag_pt_by_age["YEAR"] == year) & (chd_ed_diag_pt_by_age["AGE_GROUP"] == age_group)
            # get the population for the current county and year
            population = pop_df[(pop_df["County"] == county) & (pop_df["Year"] == year)][str(age_group)].iloc[0]
            # apply the calculation if population is not None
            if population is not None:
                chd_inp_proc_pt_by_age.loc[county_proc_mask, "POPULATION"] = population
                chd_inp_proc_pt_by_age.loc[county_proc_mask, "PROC_FRAC_BY_COUNTY"] = (chd_inp_proc_pt_by_age.loc[county_proc_mask, "PROC_COUNT"] / population)
                chd_ed_diag_pt_by_age.loc[county_diag_mask, "POPULATION"] = population
                chd_ed_diag_pt_by_age.loc[county_diag_mask, "DIAG_FRAC_BY_COUNTY"] = (chd_ed_diag_pt_by_age.loc[county_diag_mask, "DIAG_COUNT"] / population)

In [ ]:
# aggregate the inpatient data without age group
chd_inp_pt_by_age_agg = chd_inp_proc_pt_by_age.groupby(['YEAR_QTR', 'PTCOUNTY_NAME', 'RESIDENT_TYPE', "PROCEDURE"]).agg(
                                                            PROC_COUNT=pd.NamedAgg(column='PROC_COUNT', aggfunc='sum')
                                                        ).reset_index()

In [ ]:
# aggregate the inpatient data without age group
chd_ed_diag_pt_by_age_agg = chd_ed_diag_pt_by_age.groupby(['YEAR', 'PTCOUNTY_NAME', 'RESIDENT_TYPE']).agg(
                                                            DIAG_COUNT=pd.NamedAgg(column='DIAG_COUNT', aggfunc='sum'),
                                                            DIAG_FRAC_BY_COUNTY=pd.NamedAgg(column='DIAG_FRAC_BY_COUNTY', aggfunc='sum')
                                                        ).reset_index()

# Time series modeling CHD

In [ ]:
def evaluate_arima_model(X):
	""""
	Evaluate the arima model with the test dataset. train test split is 70/30
	"""
	# prepare training dataset
	train_size = int(len(X) * 0.7)
	train, test = X[0:train_size], X[train_size:]
	# make predictions
	model  = auto_arima(train, error_action="ignore", trace=False, suppress_warnings=True, maxiter=5, seasonal=True, stepwise=True)
	predictions = model.predict(n_periods=len(test))	
	# calculate out of sample error
	error_mape = mean_absolute_percentage_error(test, predictions.astype(int))
	error_mae = mean_absolute_error(test, predictions.astype(int))
	error_nmae = normalized_mean_absolute_error(test, predictions.astype(int))
	return model, error_mae, error_nmae, error_mape

def evaluate_exponential_smoothing_model(X):
	"""
	Evaluate the Holt Winter's exponential smoothing algorithm on the test dataset. train test split is 70/30
	"""
	# prepare training dataset
	train_size = int(len(X) * 0.7)
	train, test = X[0:train_size], X[train_size:]
	# make predictions
	model  = ExponentialSmoothing(np.array(train["DIAG_COUNT"]), seasonal_periods=2, trend="add", seasonal="add").fit()
	predictions = model.forecast(len(test))	
	# calculate out of sample error
	error_mape = mean_absolute_percentage_error(test["DIAG_COUNT"].to_list(), predictions.astype(int))
	error_mae = mean_absolute_error(test["DIAG_COUNT"].to_list(), predictions.astype(int))
	error_nmae = normalized_mean_absolute_error(test["DIAG_COUNT"].to_list(), predictions.astype(int))
	return model, error_mae, error_nmae, error_mape

def make_series_stationary(df, column, max_diff=1):
	stationary_df = pd.DataFrame()
	stationary_df["YEAR_QTR"] = df["YEAR_QTR"]
	series = df[column]
	for i in range(max_diff):
		result = adfuller(series.dropna(), autolag="AIC")  # Ensure to drop NA values
		if result[1] > 0.05:
			# the series is not stationary, apply differencing
			stationary_series = series.diff().dropna()
		else:
			stationary_series = series
		stationary_df[column] = stationary_series
	return stationary_df[column].fillna(method="bfill"), result[1]

def normalized_mean_absolute_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred)) / (np.max(y_true) - np.min(y_true))

In [ ]:
def evaluate_exponential_smoothing_model(train, test, cfg):
    try:
        # fit the model
        model = ExponentialSmoothing(
            train,
            seasonal_periods=cfg.get('seasonal_periods'),
            trend=cfg.get('trend'),
            seasonal=cfg.get('seasonal'),
            damped_trend=cfg.get('damped_trend', False)
        ).fit(optimized=True, remove_bias=cfg.get('remove_bias', False))
        # forecast
        predictions = model.forecast(len(test))
        # compute error metrics
        mae = mean_absolute_percentage_error(test, predictions)
        return cfg, mae
    except Exception as e:
        print(f'Error in config: {cfg} - {e}')
        return cfg, np.nan
# Grid search across combinations of configurations
def exp_smoothing_grid_search(series, cfg_list, n_test):
    # split data into train and test
    train, test = series[0:-n_test], series[-n_test:]

    # evaluate configs
    scores = [evaluate_exponential_smoothing_model(train, test, cfg) for cfg in cfg_list]
    # remove configurations with errors
    scores = [s for s in scores if not np.isnan(s[1])]

    # sort configs by error, ascending
    scores.sort(key=lambda s: s[1])

    return scores

# configuration for grid search
cfg_list = [
    {'seasonal_periods': p, 'trend': t, 'seasonal': s, 'damped_trend': d, 'use_boxcox': b, 'remove_bias': r}
    for p, t, s, d, b, r in product(
        [2, 4, 6],           # seasonal periods options
        ['add', 'mul', None], # trend options
        ['add', 'mul', None], # seasonal options
        [True, False],        # damped trend options
        [True, False],        # use boxcox options
        [True, False]         # remove bias options
    )
]

In [ ]:
chd_ed_diag_pt_by_age.to_csv("data\chd_ed_diag_pt_by_age.csv")
chd_inp_proc_pt_by_age.to_csv("data\chd_inp_proc_pt_by_age.csv")

In [ ]:
# fit arima models for the treatment procedures in each county
counties = chd_inp_pt_by_age_agg["PTCOUNTY_NAME"].unique()
procedures = chd_inp_pt_by_age_agg["PROCEDURE"].unique()

results = []
for county in counties:
    for i, treatment in enumerate(procedures):
        mask = (chd_inp_pt_by_age_agg["PTCOUNTY_NAME"] == county) & (chd_inp_pt_by_age_agg["PROCEDURE"] == treatment)
        data = chd_inp_pt_by_age_agg.loc[mask]
        if not data.empty:
            m, mae, nmae, mape = evaluate_arima_model(data["PROC_COUNT"])       
            results.append({
                "County": county,
                "Procedure": treatment,
                "Model": m,
                "MAE": mae,
                "NMAE": nmae,
                "MAPE": mape
            })     
treatment_preds_results_df = pd.DataFrame(results)    

In [ ]:
#  NOTE: commented code - age wise model fitting was not yielding good results.

# counties = chd_ed_diag_pt_by_age["PTCOUNTY_NAME"].unique()
# age_groups = chd_ed_diag_pt_by_age["AGE_GROUP"].unique()
# results = []
# for county in counties:
#     fig, axs = plt.subplots(2, 3, figsize=(20, 10))
#     axs = axs.flatten()
#     plt.suptitle(f"Diagnosis count for county: {county}", fontsize=16, y=1.02)  

#     for i, age_group in enumerate(age_groups):
#         ax = axs[i]
#         mask = (chd_ed_diag_pt_by_age["PTCOUNTY_NAME"] == county) & (chd_ed_diag_pt_by_age["AGE_GROUP"] == age_group)
#         data = chd_ed_diag_pt_by_age.loc[mask]
#         if not data.empty:
#             sns.lineplot(data=data, x="YEAR_QTR", y="DIAG_COUNT", ax=ax, label="Original")
#             sns.lineplot(data=data, x="YEAR_QTR", y="DIAG_COUNT_", ax=ax, label="Stationary", color='orange')
#             m, mae, nmae, mape = evaluate_arima_model(data["DIAG_COUNT"])       
#             results.append({
#                 "County": county,
#                 "Age_Group": age_group,
#                 "Model": m,
#                 "MAE": mae,
#                 "NMAE": nmae,
#                 "MAPE": mape
#             })

#             ax.set_title(f"{age_group} - {p}")
#             ax.set_ylabel('Diagnosis Count')
#             ax.set_xlabel('Year Quarter')
#             ax.tick_params(axis='x', rotation=90)
    
#     for j in range(i+1, 6):
#         fig.delaxes(axs[j])

#     plt.tight_layout()  # adjust the layout
#     plt.show()       
# results_df = pd.DataFrame(results)         

In [ ]:
# counties = chd_ed_diag_pt_by_age["PTCOUNTY_NAME"].unique()
# age_groups = chd_ed_diag_pt_by_age["AGE_GROUP"].unique()
# results = []
# for county in counties:
#     fig, axs = plt.subplots(2, 3, figsize=(20, 10))
#     axs = axs.flatten()
#     plt.suptitle(f"Diagnosis count for county: {county}", fontsize=16, y=1.02)  

#     for i, age_group in enumerate(age_groups):
#         ax = axs[i]
#         mask = (chd_ed_diag_pt_by_age["PTCOUNTY_NAME"] == county) & (chd_ed_diag_pt_by_age["AGE_GROUP"] == age_group)
#         data = chd_ed_diag_pt_by_age.loc[mask]
#         if not data.empty:
#             plot_acf(data["DIAG_COUNT"], ax=ax, lags=len(data)-1)
#             ax.set_title(f"ACF for {age_group}", fontsize=10)
#             ax.set_xlabel("Lags")
#             ax.set_ylable("Autocorrelation")
#         else:
#             ax.set_visible(False)
    
#     for j in range(i+1, 6):
#         fig.delaxes(axs[j])

#     plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust the layout
#     plt.show()       
# results_df = pd.DataFrame(results)         

In [ ]:
# fit arima models for the diagnostics procedures in each county
results = []
for county in chd_ed_diag_pt_by_age_agg["PTCOUNTY_NAME"].unique():
    mask =  (chd_ed_diag_pt_by_age_agg["PTCOUNTY_NAME"] == county)
    data = chd_ed_diag_pt_by_age_agg.loc[mask]
    m, mae, nmae, mape = evaluate_arima_model(data["DIAG_COUNT"])       
    results.append({
        "County": county,
        "Model": m,
        "MAE": mae,
        "NMAE": nmae,
        "MAPE": mape
    })

diagnostics_preds_results_df = pd.DataFrame(results)

In [ ]:
treatment_preds_results_df.to_excel("results-treatments-ARIMA.xlsx")
diagnostics_preds_results_df.to_excel("results-diagnostics-ARIMA.xlsx")

In [ ]:
# fit exponential smoothing algorithm on diagnostics
counties = chd_ed_diag_pt_by_age["PTCOUNTY_NAME"].unique()
results = []
for county in counties:
    mask = (chd_ed_diag_pt_by_age["PTCOUNTY_NAME"] == county)
    data = chd_ed_diag_pt_by_age.loc[mask]
    best_cfg, best_score = exp_smoothing_grid_search(data["DIAG_COUNT"], cfg_list, 4)[0]
    results.append({
            "County": county,
            "cfg": best_cfg,
            "MAPE": best_score
    })
diagnostics_preds_results_df = pd.DataFrame(results)

In [ ]:
# fit exponential smoothing algorithm on treatment procedures
counties = chd_ed_diag_pt_by_age["PTCOUNTY_NAME"].unique()
procedures = chd_inp_pt_by_age_agg["PROCEDURE"].unique()
results = []
for county in counties:
    for i, procedure in enumerate(procedures):
       mask = (chd_ed_diag_pt_by_age["PTCOUNTY_NAME"] == county) & (chd_ed_diag_pt_by_age["PROCEDURE"] == procedure)
       data = chd_ed_diag_pt_by_age.loc[mask]
       best_cfg, best_score = exp_smoothing_grid_search(data["PROC_COUNT"], cfg_list, 4)[0]
       results.append({
              "County": county,
              "cfg": best_cfg,
              "MAPE": best_score
        })
treatment_preds_results_df = pd.DataFrame(results)

In [ ]:
treatment_preds_results_df.to_excel("results-treatments-EXPONENTIAL-SMOOTHING.xlsx")
diagnostics_preds_results_df.to_excel("results-diagnostics-EXPONENTIAL-SMOOTHING.xlsx")

Note: The results-treatments-ARIMA, results-diagnostics-EXPONENTIAL-SMOOTHING and results-treatments-EXPONENTIAL-SMOOTHING, results-diagnostics-ARIMA  were combined manually on excel to filter out the best model and the resulting files were named:
1. diagnosis-model-configs.xlsx
2. treatments-model-configs.xlsx

In [ ]:
# plot a time sries plot for the diagnosis counts by year quater.
for county in chd_ed_diag_pt_by_age["PTCOUNTY_NAME"].unique():
    data = chd_ed_diag_pt_by_age[chd_ed_diag_pt_by_age["PTCOUNTY_NAME"]==county]
    plt.figure(figsize=(12, 8))
    sns.lineplot(data=data, x="YEAR_QTR", y="DIAG_COUNT", markers='o')
    plt.title(f"Diagnosis count fo county: {county}")
    plt.ylabel("diagnosis count")
    plt.xlabel("year")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Network analysis to observe the percentage of patients travelling to difference facility counties for treatment

In [ ]:
# filter only resident data and group by year again.
chd_inp_tr_by_fc = chd_inp_df.groupby(["YEAR", "PTCOUNTY_NAME", "FAC_COUNTY_NAME", "PROCEDURE", "RESIDENT_TYPE"]).size().reset_index(name="PROC_COUNT")
chd_inp_tr_by_fc = chd_inp_tr_by_fc[~(chd_inp_tr_by_fc["RESIDENT_TYPE"]=="Non Resident")]
chd_ed_diag_by_fc = chd_inp_df.groupby(["YEAR", "PTCOUNTY_NAME", "FAC_COUNTY_NAME", "RESIDENT_TYPE"]).size().reset_index(name="DIAG_COUNT")
chd_ed_diag_by_fc = chd_ed_diag_by_fc[~(chd_ed_diag_by_fc["RESIDENT_TYPE"]=="Non Resident")]

In [ ]:
for county in chd_inp_tr_by_fc["PTCOUNTY_NAME"].unique():
    for year in chd_inp_tr_by_fc["YEAR"].unique():
        chd_inp_tr_by_fc["PROC_COUNT"] =  chd_inp_tr_by_fc["PROC_COUNT"].fillna(0)
        chd_inp_tr_by_fc.loc[(chd_inp_tr_by_fc["PROCEDURE"] == "CABG")
                                    & (chd_inp_tr_by_fc["PTCOUNTY_NAME"] == county)
                                    & (chd_inp_tr_by_fc["YEAR"] == year), "PROC_COUNT"] = sum(chd_inp_tr_by_fc.loc[(chd_inp_tr_by_fc["PROCEDURE"] == "CABG + PCI")
                                                                                                                                                    & (chd_inp_tr_by_fc["PTCOUNTY_NAME"] == county)
                                                                                                                                                    & (chd_inp_tr_by_fc["YEAR"] == year), "PROC_COUNT"],
                                                                                                                        chd_inp_tr_by_fc.loc[(chd_inp_tr_by_fc["PROCEDURE"] == "CABG")
                                                                                                                                                    & (chd_inp_tr_by_fc["PTCOUNTY_NAME"] == county)
                                                                                                                                                    & (chd_inp_tr_by_fc["YEAR"] == year), "PROC_COUNT"])
        chd_inp_tr_by_fc.loc[(chd_inp_tr_by_fc["PROCEDURE"] == "PCI")
                                    & (chd_inp_tr_by_fc["PTCOUNTY_NAME"] == county)
                                    & (chd_inp_tr_by_fc["YEAR"] == year), "PROC_COUNT"] = sum(chd_inp_tr_by_fc.loc[(chd_inp_tr_by_fc["PROCEDURE"] == "CABG + PCI")
                                                                                                                                                    & (chd_inp_tr_by_fc["PTCOUNTY_NAME"] == county)
                                                                                                                                                    & (chd_inp_tr_by_fc["YEAR"] == year), "PROC_COUNT"],
                                                                                                                        chd_inp_tr_by_fc.loc[(chd_inp_tr_by_fc["PROCEDURE"] == "PCI")
                                                                                                                                                    & (chd_inp_tr_by_fc["PTCOUNTY_NAME"] == county)
                                                                                                                                                    & (chd_inp_tr_by_fc["YEAR"] == year), "PROC_COUNT"])
chd_inp_tr_by_fc = chd_inp_tr_by_fc[~((chd_inp_tr_by_fc["PROCEDURE"] == "CABG + PCI"))]

In [ ]:
grouped = chd_inp_tr_by_fc.groupby(["PTCOUNTY_NAME", "FAC_COUNTY_NAME", "PROCEDURE"]).agg(
                                                            PROC_COUNT=pd.NamedAgg(column='PROC_COUNT', aggfunc='sum')
                                                        ).reset_index()

In [ ]:

max_percentage_dict = {}
counties = chd_inp_tr_by_fc["PTCOUNTY_NAME"].unique()

for county in counties:
    max_percentage_dict[county] = {}  # Initialize a sub-dictionary for each county
    fig, axes = plt.subplots(1, len(treatments[:-1]), figsize=(20, 6))  # Create a subplot for each treatment

    for index, procedure in enumerate(treatments[:-1]):  # Iterate through treatments
        records = chd_inp_tr_by_fc.loc[(chd_inp_tr_by_fc["PTCOUNTY_NAME"] == county) & (chd_inp_tr_by_fc["PROCEDURE"] == procedure)]
        if not records.empty:
            order = records.groupby("FAC_COUNTY_NAME")['PROC_COUNT'].sum().sort_values(ascending=False).index
            sns.barplot(data=records, x="FAC_COUNTY_NAME", y="PROC_COUNT", order=order, ax=ax)
            
            total = records['PROC_COUNT'].sum()
            for p in ax.patches:
                percentage = 100 * p.get_height() / total
                x = p.get_x() + p.get_width() / 2
                y = p.get_height()
                ax.annotate(f'{percentage:.1f}%', (x, y), ha='center', va='bottom')

            # storing each facility's percentage for this procedure in the nested dictionary
            for facility in order:
                facility_total = records[records['FAC_COUNTY_NAME'] == facility]['PROC_COUNT'].sum()
                percentage = 100 * facility_total / total
                max_percentage_dict[county][procedure][facility] = round(percentage, 2)

        ax.set_title(f"Percentage of {procedure} Treatment in {county}")
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
        ax.set_xlabel("Facility County")
        ax.set_ylabel("Procedure Count")

    
    plt.tight_layout()
    plt.show()


In [ ]:
chd_inp_tr_by_fc[chd_inp_tr_by_fc["PTCOUNTY_NAME"] == "Alachua"]

In [ ]:
max_percentage_dict = {}
counties = chd_inp_tr_by_fc["PTCOUNTY_NAME"].unique()

for county in counties:
    max_percentage_dict[county] = {treatment: {} for treatment in treatments[:-1]}
    num_treatments = len(treatments[:-1])
    fig, axes = plt.subplots(1, num_treatments, figsize=(20, 6))

    for index, procedure in enumerate(treatments[:-1]):
        ax = axes[index] if num_treatments > 1 else axes
        records = grouped[(grouped["PTCOUNTY_NAME"] == county) & (grouped["PROCEDURE"] == procedure)]
        if not records.empty:
            order = records.groupby("FAC_COUNTY_NAME")['PROC_COUNT'].sum().sort_values(ascending=False).index
            sns.barplot(data=records, x="FAC_COUNTY_NAME", y="PROC_COUNT", order=order, ax=ax)
            
            total = records['PROC_COUNT'].sum()
            for p in ax.patches:
                percentage = 100 * p.get_height() / total
                x = p.get_x() + p.get_width() / 2
                y = p.get_height()
                ax.annotate(f'{percentage:.1f}%', (x, y), ha='center', va='bottom')

            # storing each facility's percentage for this procedure in the nested dictionary
            for facility in order:
                facility_total = records[records['FAC_COUNTY_NAME'] == facility]['PROC_COUNT'].sum()
                percentage = 100 * facility_total / total
                max_percentage_dict[county][procedure][facility] = round(percentage, 2)

        ax.set_title(f"Percentage of {procedure} Treatment in {county}")
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
        ax.set_xlabel("Facility County")
        ax.set_ylabel("Procedure Count")

    plt.tight_layout()
    plt.show()


In [ ]:
# get the percentage of people travelling to different facility counties to get treated
max_percentage_dict
data_for_df = []

for patient_county, procedures in max_percentage_dict.items():
    for procedure, facility_counts in procedures.items():
        data_for_df.append({
            'PATIENT_COUNTY': patient_county,
            'PROCEDURE': procedure,
            'FACILITY_COUNTY_PERCENTAGES': facility_counts 
        })

# convert the list to a DataFrame
df = pd.DataFrame(data_for_df)

# show the resulting DataFrame
df.to_excel("ptcounty-to-fccounty.xlsx")

In [ ]:
def create_procedure_graph(df, procedure_name):
    """
    Create network graph for the two treatments procedures seperately
    """
    G = nx.DiGraph()
    for index, row in df.iterrows():
        patient_county = row['PATIENT_COUNTY']
        for facility_county, percentage in row['FACILITY_COUNTY_PERCENTAGES'].items():
            if patient_county != facility_county:
                G.add_edge(patient_county, facility_county, percentage=percentage)
    
    in_degrees = G.in_degree()
    in_degree_dict = dict(in_degrees)
    max_in_degree = max(in_degree_dict.values(), default=1)
    node_sizes = [in_degree_dict.get(node, 0) * 100 for node in G.nodes()]
    node_colors = [in_degree_dict[node] / max_in_degree for node in G.nodes()]

    pos = nx.kamada_kawai_layout(G)
    
    plt.figure(figsize=(15, 15))
    # Draw nodes and edges
    nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, cmap=plt.cm.viridis)
    nx.draw_networkx_labels(G, pos, font_size=8)
    nx.draw_networkx_edges(G, pos, arrowstyle='->', arrowsize=10, edge_color='grey')

    # Get the current axes and create colorbar
    ax = plt.gca()
    sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=plt.Normalize(vmin=0, vmax=max_in_degree))
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label('In-Degree')

    plt.title(f'Patient Flow for {procedure_name} Procedures')
    plt.axis('off')
    plt.show()

In [ ]:
df_pci = df[df['PROCEDURE'] == 'PCI']
df_cabg = df[df['PROCEDURE'] == 'CABG']

create_procedure_graph(df_pci, 'CABG')
#create_procedure_graph(df_cabg, 'CABG')

# Forecasting to the next 10 years

In [ ]:
def load_sarima_model(model_config, data):
    """
    Load the SARIMA configurations and fit the model
    """
    model_config = model_config.strip()
    config_order = re.match(r"ARIMA\((\d+),(\d+),(\d+)\)\((\d+),(\d+),(\d+)\)\[(\d+)\]", model_config)
    if config_order:
        p, d, q = map(int, config_order.groups()[:3])
        P, D, Q, s = map(int, config_order.groups()[3:])

        model = auto_arima(data[:-4], error_action="ignore", trace=False, suppress_warnings=True, maxiter=5, seasonal=True, stepwise=True)
        return model

def load_exponential_smoothing_model(model_config, data):
    """
    Load the Exponential Smoothing configurations and fit the model
    """
    model_config = ast.literal_eval(model_config)
    model = ExponentialSmoothing(
        data[:-4],
        seasonal_periods=model_config.get('seasonal_periods'),
        trend=model_config.get('trend'),
        seasonal=model_config.get('seasonal'),
        damped_trend=model_config.get('damped_trend', False)).fit(optimized=True, remove_bias=model_config.get('remove_bias', True))

    return model   

def predict_and_store(model, model_type, ptcounty, periods=42, start='2023Q3'):
    """
    Predict 42 quaters which gives results until 2033 Quater 3
    """
    # create a range of future quarters
    future_quarters = pd.period_range(start=start, periods=periods, freq='Q')
    
    if model_type == "SARIMA":
        predictions = model.predict(periods)
    else:
        predictions = model.forecast(periods)
    # create a DataFrame with year-quarter as one column and the predictions as another
    forecast_df = pd.DataFrame({
        "PTCOUNTY_NAME":ptcounty,
        'YEAR_QTR': future_quarters.strftime('%YQ%q'),
        'DIAG_COUNT': predictions
    })
    return forecast_df
    

In [ ]:
# read the best model configuration files for treatments and diagnostics
diag_model_config_df = pd.read_excel("diagnosis-model-configs.xlsx")
treatment_model_confg_df = pd.read_excel("treatments-model-configs.xlsx")

In [ ]:
# load and predict diagnostics
forecasted_results_diag = pd.DataFrame()
for county in counties:
    mask = (chd_ed_diag_pt_by_age_agg["PTCOUNTY_NAME"] == county)
    data = chd_ed_diag_pt_by_age_agg[mask]
    configs = diag_model_config_df.loc[(diag_model_config_df["County"] == county), ["BEST MODEL", "SARIMA", "EXPONENTIAL SMOOTHING"]].iloc[0]
    if configs["BEST MODEL"] == "SARIMA":
        model = load_sarima_model(configs["SARIMA"], data["DIAG_COUNT"])
    else:
        model = load_exponential_smoothing_model(configs["EXPONENTIAL SMOOTHING"], data["DIAG_COUNT"])
    county_forecast = predict_and_store(model, configs["BEST MODEL"], county, start='2023Q4')
    forecasted_results_diag = pd.concat([forecasted_results_diag, county_forecast], ignore_index=True)


In [ ]:
# concatanate both historical and predicted outputs into one
forecasted_results_diag_ = pd.concat([forecasted_results_diag, chd_ed_diag_pt_by_age_agg[["PTCOUNTY_NAME", "YEAR_QTR", "DIAG_COUNT"]]], ignore_index=True)

In [ ]:
# load and predict treatments
forecasted_results_proc = pd.DataFrame()
for county in counties:
    for treatment in treatments[:-1]:
        mask = (chd_inp_pt_by_age_agg["PTCOUNTY_NAME"] == county) & (chd_inp_pt_by_age_agg["PROCEDURE"] == treatment)
        data = chd_inp_pt_by_age_agg[mask]
        configs = treatment_model_confg_df.loc[(treatment_model_confg_df["County"] == county), ["BEST MODEL", "SARIMA", "EXPONENTIAL SMOOTHING"]].iloc[0]
        if configs["BEST MODEL"] == "SARIMA":
            model = load_sarima_model(configs["SARIMA"], data["PROC_COUNT"])
        else:
            model = load_exponential_smoothing_model(configs["EXPONENTIAL SMOOTHING"], data["PROC_COUNT"])
        county_forecast = predict_and_store(model, configs["BEST MODEL"], county, treatment, start='2023Q4')
        forecasted_results_proc = pd.concat([forecasted_results_proc, county_forecast], ignore_index=True)

In [ ]:
# concatanate both historical and predicted outputs into one
forecasted_results_proc_ = pd.concat([forecasted_results_proc, chd_inp_pt_by_age_agg[["PTCOUNTY_NAME", "YEAR_QTR", "PROCEDURE", "PROC_COUNT"]]], ignore_index=True)

In [ ]:
# substring the year from the yqar quater
forecasted_results_proc_["YEAR"] = forecasted_results_proc_["YEAR_QTR"].str.extract(r'(\d{4})').astype(int)
forecasted_results_diag_["YEAR"] = forecasted_results_diag_["YEAR_QTR"].str.extract(r'(\d{4})').astype(int)

# obtain the ceiling value from the predicted output.
forecasted_results_proc_annual = forecasted_results_proc_.groupby(['PTCOUNTY_NAME', 'PROCEDURE', 'YEAR'])['PROC_COUNT'].sum().reset_index()
forecasted_results_proc_annual["PROC_COUNT"] = np.ceil(forecasted_results_proc_annual["PROC_COUNT"])
forecasted_results_diag_annual = forecasted_results_diag_.groupby(['PTCOUNTY_NAME', 'YEAR'])['DIAG_COUNT'].sum().reset_index()
forecasted_results_diag_annual["DIAG_COUNT"] = np.ceil(forecasted_results_diag_annual["DIAG_COUNT"])

In [ ]:
forecasted_results_diag_annual.to_excel("CHD-Diagnostics-Predictions.xlsx")
forecasted_results_proc_annual.to_excel("CHD-Treatments-Predictions.xlsx")

In [ ]:
# obtain predictions up until 2033. Because, after 2033 theres a steep drop in the predictions which indicates a possible overfitting scenario. 
filtered_diag_forecast = forecasted_results_diag_annual[(forecasted_results_diag_annual["YEAR"] >= 2023)
                                                        & (forecasted_results_diag_annual["YEAR"] < 2034)]
filtered_proc_forecast = forecasted_results_proc_annual[(forecasted_results_proc_annual["YEAR"] >= 2023)
                                                        & (forecasted_results_diag_annual["YEAR"] < 2034)]

# plot the predictions in a time series plot
fig, axes = plt.subplots(nrows=len(counties), ncols=len(treatments), figsize=(30, 7 * len(counties)))

for i, county in enumerate(counties):
    county_data = filtered_diag_forecast[(filtered_diag_forecast["PTCOUNTY_NAME"]==county)]
    ax = axes[i][0] if len(counties) > 1 else axes[0]
    ax.plot(county_data['YEAR'] , county_data['DIAG_COUNT'], marker='o', linestyle='-')
    ax.set_title(f'Diagnosis Counts for {county}')
    ax.set_xlabel('Year')
    ax.set_ylabel('Count')
    ax.grid(True)
    for j, procedure in enumerate(treatments[:-1]):
        if procedure == "PCI":
            div = 23
        else:
            div = 15
        county_data = filtered_proc_forecast[(filtered_proc_forecast["PTCOUNTY_NAME"]==county) & (filtered_proc_forecast["PROCEDURE"]==procedure)]
        ax = axes[i][j + 1] if len(counties) > 1 else axes[j + 1]
        ax.plot(county_data['YEAR'], county_data['PROC_COUNT'], marker='o', linestyle='-', color=treatment_colors[procedure])
        ax.set_title(f'{procedure} Counts for {county}')
        ax.set_xlabel('Year')
        ax.set_ylabel('Count')
        ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# read the percentage of people travelling to different counties for treatment.
data = pd.read_excel("ptcounty-to-fccounty.xlsx")
data = data.drop(columns="Unnamed: 0")

In [ ]:
# assuming 'data' is your initial DataFrame loaded from an Excel file

def expand_facility_county(row):
    """
    this function will parse the 'Facility County %' column which contains string representations of dictionaries
    """
    # Convert the string representation of a dictionary to an actual dictionary
    facility_county_percentages = literal_eval(row['FACILITY_COUNTY_PERCENTAGES'])
    # Create a list of tuples (Facility County, Percentage)
    return [(facility, percentage) for facility, percentage in facility_county_percentages.items()]

# apply the function to each row and store the result in a new 'expanded' column
data['expanded'] = data.apply(expand_facility_county, axis=1)

# now we create a new dataFrame to hold the expanded information
expanded_data = []

# iterate over the initial dataFrame
for index, row in data.iterrows():
    for facility, percentage in row['expanded']:
        expanded_data.append({
            'County': row['PATIENT_COUNTY'],
            'Procedure': row['PATIENT_COUNTY'],
            'Facility County': facility,
            'Percentage': percentage
        })

expanded_df = pd.DataFrame(expanded_data)
print(expanded_df)

In [ ]:
# write the exapanded dataframe to a new csv file.
expanded_df.to_excel("ptcounty-to-fccounty-expanded.xlsx")